In [20]:
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossAnharmonicity,lossTransitionFlatness
from torch.linalg import eigvalsh
from scipy.sparse.linalg import eigsh
from scipy import sparse
from torch import tensor
from numpy import arange,array
from DiSuQ.utils import plotCompare
from DevSuQ.utils import plotTrajectory
from torch import set_num_threads
set_num_threads(30)

In [15]:
Ec,Ej = 100,20
gamma = 1.5
n = 1
array_range = arange(3,4)

In [16]:
N = 3
basis = [6]+[n]*N
circuit = models.fluxoniumArray(basis,N=N,Ec=Ec,Ej=Ej,sparse=True)
H_LC = circuit.chargeHamiltonianLC()
H_J = circuit.josephsonCharge

In [17]:
H_LC.is_sparse

True

In [36]:
indices = H_LC.indices().numpy()

In [37]:
values = H_LC.values().detach().numpy()

In [53]:
H_LC.size()

torch.Size([351, 351])

In [54]:
sparse.coo_matrix((values,indices),shape=H_LC.size())

<351x351 sparse matrix of type '<class 'numpy.complex64'>'
	with 95697 stored elements in COOrdinate format>

In [ ]:
sparse.coo_matrix()

In [29]:
H_LC

tensor(indices=tensor([[  0,   0,   0,  ..., 350, 350, 350],
                       [  0,   2,   1,  ..., 344, 343, 342]]),
       values=tensor([27199.9961+0.j,     0.0000+0.j,     0.0000+0.j,  ...,
                          0.0000+0.j,     0.0000+0.j,     0.0000+0.j]),
       size=(351, 351), nnz=95697, layout=torch.sparse_coo,
       grad_fn=<DivBackward0>)

In [21]:
sparse.coo_matrix(H_LC)

TypeError: can't convert Sparse layout tensor to numpy. Use Tensor.dense() first.

In [18]:
eigsh(H_LC)

TypeError: Cannot interpret 'torch.complex64' as a data type

In [13]:
eigvalsh(H_LC.to_dense(),subset_by_index=[0,4])

TypeError: linalg_eigvalsh() got an unexpected keyword argument 'subset_by_index'

In [5]:
E0,E1,E2 = circuit.circuitEnergy(H_LC,H_J,dict()).detach().numpy()[:3]

In [6]:
E0,E1,E2

(-0.4983876, 266.16342, 266.16672)

In [ ]:
path_array, path_approx = [],[]
for N in array_range:
    basis = [6]+[n]*N
    circuit = models.fluxoniumArray(basis,N=N,Ec=Ec,Ej=Ej,sparse=False)
    H_LC = circuit.chargeHamiltonianLC()
    H_J = circuit.josephsonCharge
    E0,E1,E2 = circuit.circuitEnergy(H_LC,H_J,dict()).detach().numpy()[:3]
    path_array.append((E1-E0,E2-E1))
    del circuit
    
    El = gamma*Ej/N
    basis = {'O':[1000],'I':[],'J':[]}
    circuit = models.fluxonium(basis,El,Ec,Ej,sparse=False)
    H_LC = circuit.kermanHamiltonianLC()
    H_J = circuit.kermanHamiltonianJosephson
    E0,E1,E2 = circuit.circuitEnergy(H_LC,H_J,{'I':tensor(0.0)}).detach().numpy()[:3]
    path_approx.append((E1-E0,E2-E1))
    del circuit
    
    print(N,'-------------')

In [ ]:
path_array, path_approx = array(path_array),array(path_approx)

In [ ]:
plotTrajectory(array_range,{'array':path_array,'approx':path_approx},'Fluxonium - Quasi approximation','E10','E21',save=True)